In [ ]:
import torch
import matplotlib.pyplot as plt

In [ ]:
words = open('names.txt', 'r').read().splitlines()

chars = ['.'] + sorted(list(set(''.join(words))))
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for i, s in enumerate(chars)}

# Bigram counts.
B = torch.zeros((len(chars), len(chars)), dtype=torch.int32)
for word in words:
    chars = ['.'] + list(word) + ['.']
    for char1, char2 in zip(chars, chars[1:]):
        B[stoi[char1], stoi[char2]] += 1

In [ ]:
# Visualising the bigram tensor.
plt.figure(figsize=(16,16))
plt.imshow(B, cmap='Blues')
for i in range(B.size(0)):
    for j in range(B.size(1)):
        plt.text(j, i, itos[i] + itos[j], ha='center', va='bottom', color='gray')
        plt.text(j, i, B[i, j].item(), ha='center', va='top', color='gray')
plt.axis('off')

In [ ]:
# Add model smoothing by adding 1 to each bigram. This removes posibility of an average negative log likelihood of inf.
P = (B + 1) / B.sum(1, keepdim=True)

# Sample from the model.
out = []
idx = 0
while True:
    idx = torch.multinomial(P[idx], num_samples=1, replacement=True).item()
    out.append(itos[idx])
    if idx == 0:
        break
print(''.join(out))

In [ ]:
# Loss function. Average negative log likelihood.
log_likelihood = 0
n = 0
for word in ['fraser', 'love']:
    chars = ['.'] + list(word) + ['.']
    for char1, char2 in zip(chars, chars[1:]):
        prob = P[stoi[char1], stoi[char2]]
        log_likelihood += torch.log(prob)
        n += 1
        print(f'{char1}{char2}: {prob:.4f}')
        
print(f'Loss: {-log_likelihood / n}')